<a href="https://colab.research.google.com/github/rauledop1/TEWS/blob/main/Catastro_BN_chileipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%pip install leafmap rarfile lonboard
#%pip install lonboard

In [13]:
import duckdb
import leafmap
import rarfile
import requests
import os
import pandas as pd
import shutil

In [14]:
#download file function
def download_file(url, output_path):
    # Send a HTTP request to the specified URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the output file in binary mode and write the content
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded file to {output_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

#uncompress rar file
def extract_rar(rar_file, extract_dir):
    # Open the rar file
    with rarfile.RarFile(rar_file) as rf:
        # Extract all contents to the specified directory
        rf.extractall(extract_dir)
        print(f"Extracted {rar_file} to {extract_dir}")

def extract_table_to_dataframe(url):
    # Send an HTTP request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Use pandas to read the HTML and extract tables
        tables = pd.read_html(response.text)

        # If tables were found, return the first table as a DataFrame
        if tables:
            df = tables[0]
            print(f"Extracted table with {df.shape[0]} rows and {df.shape[1]} columns")
            return df
        else:
            print("No tables found on the webpage.")
            return None
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None

def find_shp_files(base_dir):
    shp_files = []

    # Walk through the directory and subdirectories
    for root, dirs, files in os.walk(base_dir):
        # Search for .shp files
        for file in files:
            if file.endswith('.shp'):
                shp_path = os.path.join(root, file)
                shp_files.append(shp_path)
                print(f"Found shapefile: {shp_path}")

    if shp_files:
        print(f"Found {len(shp_files)} shapefile(s).")
        return shp_files
    else:
        print("No shapefiles found.")
        return None


In [15]:
admin_url='https://raw.githubusercontent.com/rauledop1/TEWS/refs/heads/main/Catastro_BN.csv'

In [16]:
df=pd.read_csv(admin_url)

In [17]:
#df=df.query("cod_region == 'CL-MA'")
url_catastro=df.file_url.to_list()
year_update=df.year_update.to_list()
cod_region=df.cod_region.to_list()
crs_file=df.crs.to_list()

In [38]:
#Create a temporal DB
con = duckdb.connect()

#Load spatial and http extensions
con.sql('''
install httpfs;
load httpfs;
INSTALL spatial;
LOAD spatial;
''')

In [39]:
for i,j,k,c in zip(url_catastro, year_update,cod_region, crs_file):

  compress_file=f'{k}_{j}.rar'
  folder_p=f'{k}_{j}_BN'
  table_name=k.replace('-','_')+f'_{j}'

  download_file(i,compress_file)

  if not os.path.exists(folder_p):
    os.makedirs(folder_p)

  extract_rar(compress_file, folder_p)

  shp=find_shp_files(folder_p)

  shp=shp[0]

  con.sql(f'''
    create table '{table_name}' as
    SELECT *, '{table_name}' as cod_catastro, st_transform(geom, '{c}','EPSG:4326') as geo_geom
      from st_read('{shp}')
    ''')
  os.remove(compress_file)
  shutil.rmtree(folder_p)

Downloaded file to CL-AI_2011.rar
Extracted CL-AI_2011.rar to CL-AI_2011_BN
Found shapefile: CL-AI_2011_BN/11. Región de Aysén/Catastro_RV_R11_2011_v2.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-AN_2019.rar
Extracted CL-AN_2019.rar to CL-AN_2019_BN
Found shapefile: CL-AN_2019_BN/CBN_REG02_2019/CBN_REG02_2019.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-AP_2015.rar
Extracted CL-AP_2015.rar to CL-AP_2015_BN
Found shapefile: CL-AP_2015_BN/catastro_rv_r15_2015_v2/catastro_rv_r15_2015_v2.shp
Found 1 shapefile(s).
Downloaded file to CL-AT_2018.rar
Extracted CL-AT_2018.rar to CL-AT_2018_BN
Found shapefile: CL-AT_2018_BN/catastro_rv_r03_2018/catastro_rv_r03_2018.shp
Found 1 shapefile(s).


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-VS_2019.rar
Extracted CL-VS_2019.rar to CL-VS_2019_BN
Found shapefile: CL-VS_2019_BN/cbn_05reg_2019_v2/cbn_05reg_2019_v2.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-NB_2015.rar
Extracted CL-NB_2015.rar to CL-NB_2015_BN
Found shapefile: CL-NB_2015_BN/Catastro_RV_R16_2015.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [45]:
con.sql('''
SHOW ALL TABLES
''')

┌──────────┬─────────┬────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [48]:
con.sql('''with aa as (
SELECT cod_catastro, superf_ha,  geo_geom FROM CL_NB_2015
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_AI_2011
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_AN_2019
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_AP_2015
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_AT_2018
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_BI_2015
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_CO_2014
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_AR_2017
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_LI_2013
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_LL_2018
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_LR_2017
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_MA_2017
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_ML_2016
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_RM_2019
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_TA_2016
union
SELECT cod_catastro, superf_ha, geo_geom FROM CL_VS_2019
)
select cod_catastro,
count(*) as npolygon,
sum(superf_ha) as sup_c,
sum(st_area(st_transform(geo_geom,'EPSG:4326','EPSG:32718'))/10000) as sup_g
from aa
group by cod_catastro
''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┬──────────┬────────────────────┬────────────────────┐
│ cod_catastro │ npolygon │       sup_c        │       sup_g        │
│   varchar    │  int64   │       double       │       double       │
├──────────────┼──────────┼────────────────────┼────────────────────┤
│ CL_CO_2014   │    42025 │  4061627.690000006 │  4074280.125308202 │
│ CL_AI_2011   │   103165 │ 10736056.099999996 │ 10736069.988596505 │
│ CL_AN_2019   │    16724 │ 12605963.981340915 │ 12719062.451548794 │
│ CL_VS_2019   │    65344 │ 1597536.3473222095 │ 1601869.1724999126 │
│ CL_RM_2019   │    48716 │   1539631.94898864 │ 1545029.9923214463 │
│ CL_AR_2017   │   196130 │  3178217.853233667 │  3180308.143380683 │
│ CL_NB_2015   │    84031 │ 1310236.3590671052 │ 1310236.3590669618 │
│ CL_AP_2015   │     5946 │ 1694479.6499999997 │ 1707846.1031168094 │
│ CL_LL_2018   │   159146 │  4839491.890000015 │  4839492.264896011 │
│ CL_LI_2013   │    71475 │ 1634436.2599999972 │ 1638333.2723468617 │
│ CL_TA_2016   │    

In [27]:
df

,Category,cod_region,name_region,desc_region,lenaguage,year_update,crs,file_name,file_url
0,region,CL-AI,Aisén del General Carlos Ibañez del Campo,"Aysén, Aisén",es,2011,EPSG:32718,Catastro_RV_R11_2011,https://ide.minagri.gob.cl/geoweb/storage/DESC...
1,region,CL-AN,Antofagasta,NaN,es,2019,EPSG:32719,cbn_reg02_2019,https://ide.minagri.gob.cl/geoweb/storage/DESC...
2,region,CL-AP,Arica y Parinacota,NaN,es,2015,EPSG:32719,catastro_RV_R15_2015,https://ide.minagri.gob.cl/geoweb/storage/DESC...
3,region,CL-AT,Atacama,NaN,es,2018,EPSG:32719,catastro_rv_r03_2018,https://ide.minagri.gob.cl/geoweb/storage/DESC...
4,region,CL-BI,Biobío,NaN,es,2015,EPSG:32718,Catastro_RV_R08_2015,https://ide.minagri.gob.cl/geoweb/storage/DESC...
5,region,CL-CO,Coquimbo,NaN,es,2014,EPSG:32719,Catastro_RV_R04_2014,https://ide.minagri.gob.cl/geoweb/storage/DESC...
6,region,CL-AR,La Araucanía,NaN,es,2017,EPSG:32718,cbn_09reg_2017,https://ide.minagri.gob.cl/geoweb/storage/DESC...
7,region,CL-LI,Libertador General Bernardo O'Higgins,O'Higgins,es,2013,EPSG:32719,Catastro_RV_R06_2013,https://ide.minagri.gob.cl/geoweb/storage/DESC...
8,region,CL-LL,Los Lagos,NaN,es,2018,EPSG:32718,cbn_10reg_2018,https://ide.minagri.gob.cl/geoweb/storage/DESC...
9,region,CL-LR,Los Ríos,NaN,es,2017,EPSG:32718,cbn_14reg_2017,https://ide.minagri.gob.cl/geoweb/storage/DESC...
